In [4]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint

In [5]:
k = 10
genes = np.zeros(k)

In [6]:
genes

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [241]:
gamma = 0.0138172
eta = 6.205e-6
phi = 6.205e-6
k1=10e-4
k2=10e-4
r_r = 1e-1
l_r = 5000
l_p = 975


def dN_dt(X, t=0):
    #X[0] = [aminoacids](t)
    #X[1] = [mRNA](t)
    #X[2] = [ribosomes](t)
    #X[3] = [proteins](t)
    
    return np.array([
            (l_p*phi - k2*X[0]*r_r*(1-gamma))*X[2] + (l_r*eta - k1*X[0]*r_r*gamma)*X[1], 
            #rho*X[3]*r_mrna/l_mrna - xi*X[1],
            gamma*X[0]*k1*X[1]*r_r/l_r - eta*X[1],
            (1-gamma)*X[0]*k2*X[1]*r_r/l_p - phi*X[1]
    ])

def cell_generation(initial_array=np.array([10e18,5773,2.11e6]).reshape(1,-1), timesteps=2000):
    initial_aa = initial_array[-1,0]/1.12
    initial_ribosomes = initial_array[-1,1]/2
    initial_proteins = initial_array[-1,2]/2
    t = np.arange(timesteps)
    
    return np.row_stack([initial_array,
                            odeint(dN_dt, [initial_aa, initial_ribosomes,initial_proteins], t)])

def sim_cell_dynamics(n_generations, t_per_generation):
    cell = cell_generation(timesteps=t_per_generation)
    for i in range(n_generations-1):
        cell = cell_generation(cell)
        
    return cell

In [243]:
cell = sim_cell_dynamics(1,254)
aas = cell[:,0]
#mrnas = cell[:,1]
ribo = cell[:,1]
prot = cell[:,2]
plt.plot(ribo, label='Ribosomes')
plt.plot(prot, label='Proteins')
#plt.xscale('log')
plt.yscale('log')
plt.xlabel('Time')
plt.ylabel('N')
plt.xlim([0,100])
plt.legend()
plt.show()

/Users/nico/anaconda3/lib/python3.6/site-packages/scipy/integrate/odepack.py:218: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)


ValueError: all the input array dimensions except for the concatenation axis must match exactly

In [154]:
cellgeneration(initial_array=cellgeneration())

NameError: name 'cellgeneration' is not defined

In [36]:
ribo

array([1.        , 1.02040748, 1.04123138, 1.06248027, 1.0841628 ,
       1.10628781, 1.12886433, 1.15190159, 1.17540897, 1.19939609,
       1.22387272, 1.24884886, 1.27433469, 1.30034064, 1.3268773 ,
       1.3539555 , 1.38158629, 1.40978096, 1.438551  , 1.46790818,
       1.49786446, 1.52843207, 1.55962349, 1.59145145, 1.62392893,
       1.6570692 , 1.69088578, 1.72539247, 1.76060335, 1.7965328 ,
       1.83319548, 1.87060634, 1.90878067, 1.94773404, 1.98748236,
       2.02804185, 2.06942905, 2.11166087, 2.15475453, 2.19872761,
       2.24359805, 2.28938418, 2.33610469, 2.38377864, 2.4324255 ,
       2.48206514, 2.53271779, 2.58440413, 2.63714527, 2.69096271,
       2.74587843, 2.80191485, 2.85909482, 2.91744169, 2.97697927,
       3.03773186, 3.09972426, 3.16298177, 3.2275302 , 3.2933959 ,
       3.36060575, 3.42918719, 3.49916819, 3.57057734, 3.64344376,
       3.71779721, 3.79366801, 3.87108715, 3.95008621, 4.03069745,
       4.11295376, 4.19688871, 4.28253655, 4.36993226, 4.45911

In [94]:
2

2